In [51]:
from sqlalchemy import create_engine, text
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [43]:
DB_USER = os.getenv('DB_USER', 'itversity_retail_user')
DB_PASSWORD = os.getenv('DB_PASSWORD', 'itversity')
DB_HOST = os.getenv('DB_HOST', 'localhost')
DB_PORT = os.getenv('DB_PORT', '5432')
DB_NAME = os.getenv('DB_NAME','youcan')

In [44]:
DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

In [45]:
engine = create_engine(DATABASE_URL)

In [46]:
def get_cohort_data():
    
    query = """
        WITH user_cohorts AS (
            SELECT 
                user_id,
                to_char(DATE_TRUNC('month', signup_date)::timestamp,'yyyy-MM') as cohort_month
            FROM users
        ),
        user_activity AS (
            SELECT 
                u.user_id,
                u.cohort_month,
                DATE_TRUNC('week', e.timestamp) as activity_week
            FROM user_cohorts u
            JOIN events e ON u.user_id = e.user_id
            GROUP BY u.user_id, u.cohort_month, DATE_TRUNC('week', e.timestamp)
        )
        SELECT 
            cohort_month,
            activity_week,
            COUNT(DISTINCT user_id) as active_users
        FROM user_activity
        GROUP BY cohort_month, activity_week
        ORDER BY cohort_month, activity_week;
    """
    
    with engine.connect() as conn:
        data = pd.read_sql(text(query),conn)
        
    return data

In [47]:
data = get_cohort_data()

In [48]:
data

,cohort_month,activity_week,active_users
0,2024-06,2024-06-10,55
1,2024-06,2024-06-17,118
2,2024-06,2024-06-24,179
3,2024-06,2024-07-01,169
4,2024-06,2024-07-08,148
...,...,...,...
145,2025-05,2025-06-02,197
146,2025-05,2025-06-09,160
147,2025-06,2025-05-26,5
148,2025-06,2025-06-02,50


In [49]:
def get_cohort_retention():
    query = text("""
        WITH user_cohorts AS (
            SELECT 
                user_id,
                to_char(DATE_TRUNC('month', signup_date)::timestamp,'yyyy-MM') as cohort_month,
                signup_date
            FROM users
        ),
        cohort_sizes AS (
            SELECT 
                cohort_month,
                COUNT(DISTINCT user_id) as total_users
            FROM user_cohorts
            GROUP BY cohort_month
        ),
        user_activity AS (
            SELECT 
                u.user_id,
                u.cohort_month,
                u.signup_date,
                DATE_TRUNC('week', e.timestamp) as activity_week,
                FLOOR(EXTRACT(EPOCH FROM (DATE_TRUNC('week', e.timestamp) - u.signup_date)) / (7 * 24 * 60 * 60)) as week_number
            FROM user_cohorts u
            JOIN events e ON u.user_id = e.user_id
            WHERE FLOOR(EXTRACT(EPOCH FROM (DATE_TRUNC('week', e.timestamp) - u.signup_date)) / (7 * 24 * 60 * 60)) >= 0
            AND FLOOR(EXTRACT(EPOCH FROM (DATE_TRUNC('week', e.timestamp) - u.signup_date)) / (7 * 24 * 60 * 60)) < 8
            GROUP BY u.user_id, u.cohort_month, u.signup_date, DATE_TRUNC('week', e.timestamp)
        )
        SELECT 
            ua.cohort_month,
            ua.week_number,
            COUNT(DISTINCT ua.user_id) as active_users,
            cs.total_users
        FROM user_activity ua
        JOIN cohort_sizes cs ON ua.cohort_month = cs.cohort_month
        GROUP BY ua.cohort_month, ua.week_number, cs.total_users
        ORDER BY ua.cohort_month, ua.week_number;
    """)
    
    with engine.connect() as conn:
        data = pd.read_sql(query, conn)
    
    retention_matrix = data.pivot(
        index='cohort_month',
        columns='week_number',
        values='active_users'
    ).fillna(0)
    
    cohort_sizes = retention_matrix[0] 
    retention_percentages = retention_matrix.div(cohort_sizes, axis=0) * 100
    
    return retention_matrix, retention_percentages

retention_matrix, retention_percentages = get_cohort_retention()


In [50]:
retention_percentages

week_number,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0
cohort_month,,,,,,,,
2024-06,100.0,88.829787,77.659574,57.446809,47.340426,35.106383,25.000000,13.297872
2024-07,100.0,87.924528,73.207547,58.867925,43.018868,30.943396,20.377358,10.943396
2024-08,100.0,89.530686,77.256318,65.342960,44.404332,29.602888,19.855596,12.996390
2024-09,100.0,92.075472,79.622642,61.509434,49.433962,36.603774,25.660377,11.320755
2024-10,100.0,88.846154,73.461538,60.000000,41.153846,26.923077,15.384615,8.461538
2024-11,100.0,89.763780,77.559055,60.236220,44.881890,30.314961,19.685039,11.811024
2024-12,100.0,90.944882,75.984252,61.023622,46.062992,33.858268,22.834646,11.023622
2025-01,100.0,86.328125,72.656250,57.031250,44.140625,30.078125,16.406250,10.156250
2025-02,100.0,88.557214,71.641791,56.218905,39.303483,26.368159,15.422886,7.462687


In [52]:
def plot_retention_heatmap(retention_matrix):
    plt.figure(figsize=(12, 8))
    
    sns.heatmap(
        retention_matrix,
        annot=True,
        fmt='.1f',
        cbar_kws={'label': 'Retention Rate (%)'}
    )
    
    plt.title('User Retention by Cohort')
    plt.xlabel('Week Number')
    plt.ylabel('Cohort Month')
    
    plt.xticks(rotation=0)
    
    plt.tight_layout()
    plt.savefig('retention_heatmap.png')
    plt.close()

In [54]:
plot_retention_heatmap(retention_percentages)

In [58]:
def plot_retention_curves(retention_matrix):
    plt.figure(figsize=(12, 6))
    
    for cohort in retention_matrix.index:
        plt.plot(
            retention_matrix.columns,
            retention_matrix.loc[cohort],
            marker='o',
            label=cohort 
        )
    
    plt.title('Retention Curves by Cohort')
    plt.xlabel('Week Number')
    plt.ylabel('Retention Rate (%)')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend(title='Cohort Month', bbox_to_anchor=(1.05, 1), loc='upper left')
    
    plt.tight_layout()
    plt.savefig('retention_curves.png')
    plt.close()

In [60]:
plot_retention_curves(retention_percentages)

In [70]:
def generate_cohort_report(retention_matrix):
    # Create the report content
    report = "Cohort Analysis Report\n"
    report += "===================\n\n"
    
    # Overall retention metrics
    report += "Overall Retention Metrics:\n"
    report += f"Average 8-week retention: {retention_matrix[7].mean():.1f}%\n"
    report += f"Median 8-week retention: {retention_matrix[7].median():.1f}%\n\n"
    
    # Best performing cohort
    best_cohort = retention_matrix[7].idxmax()
    report += f"Best performing cohort: {best_cohort}\n"
    report += f"8-week retention: {retention_matrix[7].max():.1f}%\n\n"
    
    # Trends analysis
    report += "Retention Trends:\n"
    for week in retention_matrix.columns:
        avg_retention = retention_matrix[week].mean()
        report += f"Week {int(week)} average retention: {avg_retention:.1f}%\n"
    
    report_dir = 'cohort_analysis_report'  
    os.makedirs(report_dir, exist_ok=True)
    
    report_path = os.path.join(report_dir, 'cohort_analysis_report.txt')
    with open(report_path, 'w') as f:
        f.write(report)
    
    print(f"Report saved to: {report_path}")
    return report

In [71]:
generate_cohort_report(retention_percentages)

Report saved to: cohort_analysis_report\cohort_analysis_report.txt


'Cohort Analysis Report\n===================\n\nOverall Retention Metrics:\nAverage 8-week retention: 9.1%\nMedian 8-week retention: 10.9%\n\nBest performing cohort: 2024-06\n8-week retention: 13.3%\n\nRetention Trends:\nWeek 0 average retention: 100.0%\nWeek 1 average retention: 83.5%\nWeek 2 average retention: 68.7%\nWeek 3 average retention: 53.4%\nWeek 4 average retention: 38.6%\nWeek 5 average retention: 26.5%\nWeek 6 average retention: 16.7%\nWeek 7 average retention: 9.1%\n'